In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/homelessness-data-collection/zillow_house_prices.csv
/kaggle/input/homelessness-data-collection/pit_data_interpolated.csv
/kaggle/input/homelessness-data-collection/yearly_employment_rates.csv
/kaggle/input/homelessness-data-collection/__results__.html
/kaggle/input/homelessness-data-collection/zillow_home_price_index.csv
/kaggle/input/homelessness-data-collection/unified_data.csv
/kaggle/input/homelessness-data-collection/census_poverty_income.csv
/kaggle/input/homelessness-data-collection/__notebook__.ipynb
/kaggle/input/homelessness-data-collection/state_abbrevations.csv
/kaggle/input/homelessness-data-collection/__output__.json
/kaggle/input/homelessness-data-collection/yearly_house_rents.csv
/kaggle/input/homelessness-data-collection/custom.css


In [2]:
DATA_DIR = "/kaggle/input/homelessness-data-collection"
filepath = f"{DATA_DIR}/unified_data.csv"

data_df = pd.read_csv(filepath)
data_df.shape, data_df.columns

((8007, 11),
 Index(['Date', 'Overall Homeless', 'State', 'State_Name', 'unemploy_rate',
        'employed_pop_rate', 'HomeValueIndex', 'MHHI', 'Poverty_Count',
        'Poverty_Rate', 'Min_Rent'],
       dtype='object'))

In [3]:
data_df['HomeValueIndex'] = (data_df.groupby('State')['HomeValueIndex']
                             .apply(lambda x:x.fillna(x.mean())))
data_df['MHHI'] = (data_df.groupby('State')['MHHI']
                             .apply(lambda x:x.fillna(x.mean())))
data_df['Poverty_Count'] = (data_df.groupby('State')['Poverty_Count']
                             .apply(lambda x:x.fillna(x.mean())))
data_df['Poverty_Rate'] = (data_df.groupby('State')['Poverty_Rate']
                             .apply(lambda x:x.fillna(x.mean())))
data_df['Date'] = pd.to_datetime(data_df.Date)

In [4]:
data_df['Date'].min(), data_df['Date'].max()

(Timestamp('2007-01-31 00:00:00'), Timestamp('2020-01-31 00:00:00'))

In [5]:
tx_data = data_df[data_df['State'].isin(["TX"])]
tx_data = tx_data.drop(columns=['employed_pop_rate', 'MHHI',
                                'Poverty_Count'])
tx_data.shape, tx_data.columns

((157, 8),
 Index(['Date', 'Overall Homeless', 'State', 'State_Name', 'unemploy_rate',
        'HomeValueIndex', 'Poverty_Rate', 'Min_Rent'],
       dtype='object'))

In [6]:
from sklearn.model_selection import TimeSeriesSplit
tx_train = tx_data[tx_data['Date'] <= "2018-01-31"]
tx_val = tx_data[tx_data['Date'] > "2018-01-31"]
tx_train = tx_train.set_index('Date').select_dtypes([float, int])
tx_val = tx_val.set_index('Date').select_dtypes([float, int])
tx_train.shape, tx_val.shape

((133, 5), (24, 5))

In [7]:
from statsmodels.tsa.stattools import adfuller

def adjust(val, length= 6): return str(val).ljust(length)

def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")    


tx_train_diff = tx_train.diff().dropna()
tx_train_diff2 = tx_train_diff.diff().dropna()

# time series is stationary at diff2
for name, column in tx_train_diff2.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "Overall Homeless" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -10.93
 No. Lags Chosen       = 0
 Critical value 1%     = -3.482
 Critical value 5%     = -2.884
 Critical value 10%    = -2.579
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "unemploy_rate" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -8.1326
 No. Lags Chosen       = 3
 Critical value 1%     = -3.483
 Critical value 5%     = -2.885
 Critical value 10%    = -2.579
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "HomeValueIndex" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.


In [8]:
from statsmodels.tsa.vector_ar.var_model import VAR
model = VAR(tx_train_diff2)
for i in [1,2,3,4,5]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1
AIC :  15.79344098628212
BIC :  16.455179705618022
FPE :  7230141.212416194
HQIC:  16.06232754130734 

Lag Order = 2
AIC :  15.683710716741588
BIC :  16.903010579066333
FPE :  6490087.229333459
HQIC:  16.179136934544694 

Lag Order = 3
AIC :  15.366329809955902
BIC :  17.148848724905662
FPE :  4746423.530022407
HQIC:  16.090576345547962 

Lag Order = 4
AIC :  15.481192993405738
BIC :  17.832686253863628
FPE :  5370951.208402095
HQIC:  16.436576331382714 

Lag Order = 5
AIC :  15.55896274243784
BIC :  18.485285344848094
FPE :  5891506.840631513
HQIC:  16.747835904038137 



/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  % freq, ValueWarning)


In [9]:
x = model.select_order(10)
x.summary()

,AIC,BIC,FPE,HQIC
0,16.71,16.83,1.816e+07,16.76
1,16.10,16.79*,9.816e+06,16.38*
2,15.99,17.26,8.812e+06,16.51
3,15.64*,17.49,6.229e+06*,16.39
4,15.72,18.14,6.816e+06,16.70
5,15.75,18.75,7.166e+06,16.97
6,15.95,19.53,8.969e+06,17.41
7,16.10,20.26,1.075e+07,17.79
8,16.22,20.96,1.278e+07,18.15
9,16.49,21.80,1.766e+07,18.64


In [10]:
model_fitted = model.fit(2)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 30, Oct, 2021
Time:                     17:50:25
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:                    16.9030
Nobs:                     129.000    HQIC:                   16.1791
Log likelihood:          -1871.81    FPE:                6.49009e+06
AIC:                      15.6837    Det(Omega_mle):     4.31081e+06
--------------------------------------------------------------------
Results for equation Overall Homeless
                         coefficient       std. error           t-stat            prob
--------------------------------------------------------------------------------------
const                       0.846601         6.590705            0.128           0.898
L1.Overall Homeless         0.052450         0.097134            0.540           0.589
L1.unemploy_rate         

In [11]:
# from statsmodels.stats.stattools import durbin_watson
# out = durbin_watson(model_fitted.resid)

# for col, val in zip(tx_train_diff2.columns, out):
#     print(adjust(col), ':', round(val, 2))

In [12]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)

# Input data for forecasting
forecast_input = tx_train_diff2.values[-lag_order:]
forecast_input

2


array([[   4.,    0., -162.,    0.,   -1.],
       [   1.,    0.,  -11.,    0.,    1.]])

In [13]:
nobs = 24
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc,
                           columns=tx_train_diff2.columns + '_2d')
df_forecast.columns

Index(['Overall Homeless_2d', 'unemploy_rate_2d', 'HomeValueIndex_2d',
       'Poverty_Rate_2d', 'Min_Rent_2d'],
      dtype='object')

In [14]:
def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns =[c.replace("_2d", "") for c in df_forecast.columns]
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = ((df_train[col].iloc[-1]- 
                                     df_train[col].iloc[-2]) \
                                     + df_fc[str(col)+'_2d'].cumsum())
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = (df_train[col].iloc[-1] + 
                                       df_fc[str(col)+'_1d'].cumsum())
        
    fcol = [c for c in df_fc.columns if c.endswith("forecast")]
    return df_fc[fcol]

In [15]:
df_results  = invert_transformation(tx_train, df_forecast, True)
df_results.columns

Index(['Overall Homeless_forecast', 'unemploy_rate_forecast',
       'HomeValueIndex_forecast', 'Poverty_Rate_forecast',
       'Min_Rent_forecast'],
      dtype='object')

In [16]:
from statsmodels.tsa.stattools import acf

def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    mae = np.mean(np.abs(forecast - actual))    # MAE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    return({'mape':mape, 'mae': mae, 
            'rmse':rmse, 'corr':corr})


for col in df_results.columns:
    cn = col.replace("_forecast", "")
    print('Forecast Accuracy of:', cn)
    accuracy_prod = forecast_accuracy(df_results[col].values, 
                                      tx_val[cn][:nobs].values)
    for k, v in accuracy_prod.items():
        print(adjust(k), ': ', round(v,4))
    print("*"*25)

Forecast Accuracy of: Overall Homeless
mape   :  0.0438
mae    :  1153.3754
rmse   :  1274.7053
corr   :  0.981
*************************
Forecast Accuracy of: unemploy_rate
mape   :  0.1237
mae    :  0.4423
rmse   :  0.493
corr   :  -0.7324
*************************
Forecast Accuracy of: HomeValueIndex
mape   :  0.0192
mae    :  2554.4075
rmse   :  2864.4033
corr   :  0.9773
*************************
Forecast Accuracy of: Poverty_Rate
mape   :  0.0693
mae    :  1.0715
rmse   :  1.1484
corr   :  0.7709
*************************
Forecast Accuracy of: Min_Rent
mape   :  0.0146
mae    :  8.7796
rmse   :  9.7134
corr   :  0.9975
*************************
